### Pre-requisite:
Before running this notebook, you will have to:
1. download the csv file named `dht_1k.csv` and `sds_1k.csv`  
stored under https://github.com/IBMProjectEventStore/db2eventstore-IoT-Analytics/tree/master/data.
2. Go to the `Project tab` and load both above mentioned csv files into the current project as dataset.
----
**Note: This Notebook can only run in Python version >= 3.0**

In [ ]:
from eventstore.oltp import EventContext
from eventstore.sql import EventSession
from eventstore.common import ConfigurationReader
from pyspark.sql import SparkSession

ConfigurationReader.setEventUser("")
ConfigurationReader.setEventPassword("")

In [ ]:
sparkSession = SparkSession.builder.appName("EventStore SQL in Python").getOrCreate()
eventSession = EventSession(sparkSession.sparkContext, "EVENTDB")
eventSession.set_query_read_option("SnapshotNow")
eventSession._jvm.org.apache.spark.sql.types.SqlTimeSeries.register(eventSession._jsparkSession)
eventSession.open_database()
ctx = EventContext.get_event_context("EVENTDB")

In [ ]:
from eventstore.catalog import TableSchema
from pyspark.sql.types import *

In [ ]:
table_names = ctx.get_names_of_tables()
for idx, name in enumerate(table_names):
    print(idx, name)

In [ ]:
from datetime import datetime

def datetime_converter(datetime_string):
    # (1) Convert to datetime format
    utc_time = datetime.strptime(datetime_string.split('.000Z')[0], "%Y-%m-%dT%H:%M:%S")

    return int((utc_time - datetime(1970, 1, 1)).total_seconds())

In [ ]:
## create table and loading data for DHT

In [ ]:
# Define table schema to be created
with EventContext.get_event_context("EVENTDB") as ctx:
    schema = StructType([
        StructField("sensor_id", IntegerType(), nullable = False),
        StructField("timestamp", IntegerType(), nullable = False),
        StructField("location", IntegerType(), nullable = False),
        StructField("humidity", FloatType(), nullable = True)
    ])  
    table_schema = TableSchema("dht_table", schema,
                                sharding_columns=["sensor_id"],
                                pk_columns=["sensor_id","timestamp","location"])

In [ ]:
# try create table if not exist
# try:
#     ctx.drop_table("DHT_TABLE")
# except Exception as error:
#     print(error)
try:
    ctx.create_table(table_schema)
except Exception as error:
    pass
    
table_names = ctx.get_names_of_tables()
for idx, name in enumerate(table_names):
    print(name)

In [ ]:
dht_table = eventSession.load_event_table("dht_table")

In [ ]:
# ingest data into table
import os
resolved_table_schema = ctx.get_table("dht_table")
print(resolved_table_schema)
with open(os.environ['DSX_PROJECT_DIR']+'/datasets/dht_1k.csv') as f:
    f.readline()
    content = f.readlines()
content = [l.split(",") for l in content]
batch = [dict(sensor_id=int(c[5]), timestamp=datetime_converter(c[7]), location=int(c[0]), humidity=float(c[2])) for c in content]
ctx.batch_insert(resolved_table_schema, batch)

In [ ]:
# verify ingested result
dht_table = eventSession.load_event_table("dht_table")

In [ ]:
dht_table.createOrReplaceTempView("dht_raw_table")

In [ ]:
eventSession.sql("select count(*) from dht_raw_table").show()

In [ ]:
## create table and loading data for SDS

In [ ]:
with EventContext.get_event_context("EVENTDB") as ctx:
    schema = StructType([
        StructField("sensor_id", IntegerType(), nullable = False),
        StructField("timestamp", LongType(), nullable = False),
        StructField("location", IntegerType(), nullable = False),
        StructField("p_1", DoubleType(), nullable = True)
    ])  
    table_schema = TableSchema("sds_table", schema,
                                sharding_columns=["sensor_id"],
                                pk_columns=["sensor_id","timestamp","location"])


In [ ]:
# try:
#     ctx.drop_table("SDS_TABLE")
# except Exception as error:
#     print(error)
try:
    ctx.create_table(table_schema)
except Exception as error:
    print("Table not created.")
table_names = ctx.get_names_of_tables()
for idx, name in enumerate(table_names):
    print(name)

In [ ]:
sds_table = eventSession.load_event_table("sds_table")

In [ ]:
with EventContext.get_event_context("EVENTDB") as ctx:
    resolved_table_schema = ctx.get_table("sds_table")
    with open(os.environ['DSX_PROJECT_DIR']+'/datasets/sds_1k.csv') as f:
        f.readline()
        content = f.readlines()
    content = [l.split(",") for l in content]
    batch = [dict(sensor_id=int(c[5]), timestamp=datetime_converter(c[7]), location=int(c[0]), p_1=float(c[2])) for c in content if c[2] != ""]
    ctx.batch_insert(resolved_table_schema, batch)

In [ ]:
sds_table=eventSession.load_event_table("sds_table")
sds_table.count()

In [ ]:
sds_table.createOrReplaceTempView("sds_raw_table")

In [ ]:
eventSession.sql("select * from sds_raw_table").show(5)

In [ ]:
## Query

In [ ]:
sql="SELECT count(*) FROM dht_raw_table"
eventSession.sql(sql).show(5)

# Alignment of multiple time-series from different IoT sensors

The below example shows how one can align two timeseries using an inner join with the nearest timestamp. The problem arises when two sensors are generating timestamps with clocks that are not fully synchronized. This can occur regularly and is common in IoT scenarios.

In what follows, we will show an example on the OK Lab data that considers the timeseries generated by the SDS (particulate matter) sensors and the DHT (humidity/temperature) sensors. The manual of SDS states that the SDS values are valid only when the DHT sensor measures humidity as < 70%. Realizing this restraint requires the data from the 2 sensors to be synchronized temporally. We note first that traditional join style would be to look for exact timestamps, which will not work in this case as the sensors are coming from two different devices (with the same location). One approach would be to use a standard SQL statement such as:

```sql
SELECT 
    humidity, 
    p_1
FROM sds_raw_table STORED AS PARQUET sds011, dht_raw_table STORED AS PARQUET dht22
AND dht22.humidity <= 70
AND ((sds011.timestamp - INTERVAL 10 SECONDS) < dht22.timestamp) 
AND (dht22.timestamp < (sds011.timestamp + INTERVAL 10 SECONDS))
```

However, if executed as it stands, it can take several hours to complete given that it results in a traditional Cartesian Join.

In our example, we will down-select the data using DB2 Eventstore into two dataframes (sds and dht). Our approach then is to do a clever join that takes windowing into account and not do a full Cartesian join. 

In what follows, we we will show how using time-series capabilities of DB2-Eventstore can be used to address this problem of unaligned sensors. Although this is one application that time-series for DB2-eventstore covers, time-series capabilites are not limited to just this one use case as we have functions to handle simple statistical methods (fft, avg, percentile, etc.), time-series distance metrics (DL, DTW, SBD, etc.), sophisticated forms of segmentation (time-based, anchor-base, marker-base, record-based, etc.), etc.

# Initial exploration on data

In many cases, the first step to doing any Time-Series analysis is to learn about your time-series. To do so, we use what is called a describe. This will provide a rich set of metrics (avg, percentiles, timing-statistics, etc.) over a time-series such that a user can have some knowledge of the time-series they are working with.

In [ ]:
stmt = """SELECT location, TS_DESCRIBE(ts) FROM ( SELECT location,TIME_SERIES(timestamp, humidity) AS ts FROM dht_raw_table where humidity < 70 GROUP BY location)
"""

df = eventSession.sql(stmt).toPandas()
df.head()

## Description of performing a time-series sql temporal align

This query has a few main things to consider:

### Creating your time series

first a time series must be created

#### sds time series

```sql
SELECT location, D_TIME_SERIES(timestamp, p_1) AS sds FROM sds_raw_table GROUP BY location
```

#### dht time series

```sql
SELECT location, D_TIME_SERIES(timestamp, humidity) AS dht FROM dht_raw_table GROUP BY location
```

### Performing full temporal align

Performing a full temporal align requires 2 parameters:

- The left Time Series
- The right Time Series

Once given, the returned output will be 2 columns (the 2 aligned time series) as **left_column_aligned** and **right_column_aligned**

*Note: With this method, all missing values will be replaced with null*

In [ ]:
stmt = """
    SELECT sds_table.location, TS_FULL_ALIGN(dht, sds, TS_INTERPOLATOR_NEAREST(-1.0)) FROM 
        (SELECT location, TIME_SERIES(timestamp, humidity) AS dht FROM dht_raw_table where humidity < 70 GROUP BY location) AS dht_table
        INNER JOIN
        (SELECT location, TIME_SERIES(timestamp, p_1) AS sds FROM sds_raw_table GROUP BY location) AS sds_table
        ON dht_table.location = sds_table.location
"""
df = eventSession.sql(stmt)
df.show()
df.count()
eventSession.sql(stmt).createOrReplaceTempView("dht_sds_ts_table")

# Display the aligned TimeSeries table

In [ ]:
eventSession.sql("select count(*) from dht_sds_ts_table").show()

# Interpolate missing values after alignment

Because in IoT use cases, sensors tend to be clocked at different rates, it's important to properly fill values where they don't exist in the data. Just because a value is not in our data, does not mean it did not exist. To approximate the missing value, we can provide an interpolator as simple as nearest, next, prev, but as sophisticated as linear interpolation or cubic spline interpolation. In the following example, we will fill all missing values based on a nearest interpolation method.

In [ ]:
eventSession.sql("SELECT location, TS_FILLNA(dht_aligned,TS_INTERPOLATOR_NEAREST(-1.0)) as ts FROM dht_sds_ts_table").createOrReplaceTempView("dht_no_nulls")
eventSession.sql("SELECT location, TS_FILLNA(sds_aligned,TS_INTERPOLATOR_NEAREST(-1.0)) as ts FROM dht_sds_ts_table").createOrReplaceTempView("sds_no_nulls")

# Converting Time-Series data to tabular data

Once all Time-Series analysis has been done, because Time-Series types are not directly ingestable, a user may want to display there data in a tabular format to prepare for graphing or performing further analysis. The following is how would could convert that data.

In [ ]:
eventSession.sql("SELECT location, TS_EXPLODE(ts) FROM dht_no_nulls").createOrReplaceTempView("dht_exploded")
eventSession.sql("SELECT location, TS_EXPLODE(ts) FROM sds_no_nulls").createOrReplaceTempView("sds_exploded")

In [ ]:
eventSession.sql("select * from dht_exploded").count()

In [ ]:
eventSession.sql("select * from sds_exploded").count()

# Joining the tabular data

Lastly, we will perform a classical join on the location and time_tick for humidity and coarse particulate matter data to properly display the aligned values

In [ ]:
stmt = """
    select dht_exploded.location, dht_exploded.ts_timeTick as timestamp, dht_exploded.ts_value as humidity, sds_exploded.ts_value as p_1 FROM
        dht_exploded
        INNER JOIN
        sds_exploded
        ON dht_exploded.location=sds_exploded.location and dht_exploded.ts_timeTick=sds_exploded.ts_timeTick
"""
df = eventSession.sql(stmt)
df.show()
df.count()

## Timing for Time-Series SQL
This section shows how long it takes to run each time-series SQL query.

In [ ]:
import time

stmt = "SELECT TIME_SERIES(timestamp, humidity) AS dht FROM dht_raw_table"

df = eventSession.sql(stmt)
df.cache()
df.createOrReplaceTempView("table_a")
start = time.time()
df.show()
end = time.time()

In [ ]:
total_time = end - start
print("ingestion time: " + str(total_time) + " seconds")

In [ ]:
stmt = "SELECT TS_RESAMPLE(dht, 3600, TS_INTERPOLATOR_NEAREST(0.0)) as dht_interp from table_a"
start = time.time()
eventSession.sql(stmt).show()
end = time.time()

In [ ]:
total_time = end - start
print("ingestion time: " + str(total_time) + " seconds")

In [ ]:
stmt = "select count(*) from (SELECT TS_EXPLODE(dht) from table_a)"
start = time.time()
eventSession.sql(stmt).show()
end = time.time()

In [ ]:
total_time = end - start
print("ingestion time: " + str(total_time) + " seconds")